In [ ]:
#https://www.kaggle.com/c/invasive-species-monitoring/data

In [1]:
from datetime import datetime
HOMEPATH = "/home/ubuntu/fastai/"
DATA_HOME_DIR = HOMEPATH + "data/invasive/"

validFract = 0.3
sampleFract = 0.2
import os

In [2]:
os.chdir(HOMEPATH)
print ("os.getcwd:", os.getcwd())
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
import utils;
from utils import *
from shutil import copyfile

from __future__ import division, print_function

from vgg16 import Vgg16
from keras.models import load_model

from theano.sandbox import cuda
cuda.use('gpu0')


import numpy as np
import pandas as pd
from random import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

('os.getcwd:', '/home/ubuntu/fastai')


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [3]:
#model training configuration
batch_size = 64

In [4]:
TRAIN  = DATA_HOME_DIR + "train/"
VALID  = DATA_HOME_DIR + "valid/"
TEST   = DATA_HOME_DIR + "test/"
RESULTS = DATA_HOME_DIR + "results/"
SAMPLE =  DATA_HOME_DIR + "sample/"
SAMPLE_TRAIN = SAMPLE + "train/"
SAMPLE_VALID = SAMPLE + "valid/"
SAMPLE_TEST = SAMPLE + "test/"
SAMPLE_RESULTS = SAMPLE + "results/"

In [58]:
#change this as appropriate between smaller sample data set(SAMPLE) and production data(DATA_HOME_DIR)
WORKING_DATA = SAMPLE
#WORKING_DATA = DATA_HOME_DIR 
#------------------future me : swap between above options. do not change below--------------
WORKING_TEST = WORKING_DATA + "test/" #We use all the test data
WORKING_TRAIN = WORKING_DATA + "train/"
WORKING_VALID = WORKING_DATA + "valid/"
WORKING_RESULTS = WORKING_DATA + "results/"

print ("WORKING_DATA:", WORKING_DATA)
print ("WORKING_TEST:", WORKING_TEST)
print ("WORKING_TRAIN:", WORKING_TRAIN)
print ("WORKING_VALID:", WORKING_VALID)
print ("WORKING_RESULTS:", WORKING_RESULTS)
model_path = WORKING_RESULTS
print ("model_path:", model_path)

WORKING_DATA: /home/ubuntu/fastai/data/invasive/sample/
WORKING_TEST: /home/ubuntu/fastai/data/invasive/sample/test/
WORKING_TRAIN: /home/ubuntu/fastai/data/invasive/sample/train/
WORKING_VALID: /home/ubuntu/fastai/data/invasive/sample/valid/
WORKING_RESULTS: /home/ubuntu/fastai/data/invasive/sample/results/
model_path: /home/ubuntu/fastai/data/invasive/sample/results/


In [6]:
# traverse root directory, and list directories as dirs and files as files
print (DATA_HOME_DIR)
import os
next(os.walk(DATA_HOME_DIR))[1]
os.chdir(DATA_HOME_DIR)
print ("os.getcwd:", os.getcwd())
!tree -d

/home/ubuntu/fastai/data/invasive/
os.getcwd: /home/ubuntu/fastai/data/invasive
.
├── results
├── sample
│   ├── results
│   ├── test
│   │   └── unknown
│   ├── train
│   │   ├── harmless
│   │   └── invasive
│   └── valid
│       ├── harmless
│       └── invasive
├── test
│   └── unknown
├── train
│   ├── harmless
│   └── invasive
└── valid
    ├── harmless
    └── invasive

19 directories


In [19]:
print ("WORKING_TRAIN:", WORKING_TRAIN)
print ("WORKING_VALID:", WORKING_VALID)
batches = get_batches(WORKING_TRAIN, shuffle=False, batch_size=batch_size)
val_batches = get_batches(WORKING_VALID, shuffle=False, batch_size=batch_size)

WORKING_TRAIN: /home/ubuntu/fastai/data/invasive/sample/train/
WORKING_VALID: /home/ubuntu/fastai/data/invasive/sample/valid/
Found 320 images belonging to 2 classes.
Found 136 images belonging to 2 classes.


In [8]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filenames) = get_classes(WORKING_DATA)

Found 320 images belonging to 2 classes.
Found 136 images belonging to 2 classes.
Found 306 images belonging to 1 classes.


In [11]:
print ("val_classes:", type(val_classes), val_classes.shape)
print ("trn_classes:", type(trn_classes), trn_classes.shape)
print ("val_labels:", type(val_labels), val_labels.shape)
print ("trn_labels:", type(trn_labels), trn_labels.shape)
print ("val_filenames:", type(val_filenames), len(val_filenames))
print ("filenames:", type(filenames), len(filenames))
print ("test_filenames:", type(test_filenames), len(test_filenames))

val_classes: <type 'numpy.ndarray'> (136,)
trn_classes: <type 'numpy.ndarray'> (320,)
val_labels: <type 'numpy.ndarray'> (136, 2)
trn_labels: <type 'numpy.ndarray'> (320, 2)
val_filenames: <type 'list'> 136
filenames: <type 'list'> 320
test_filenames: <type 'list'> 306


In [61]:
model = Vgg16().model
#NB: utils.split_at returns two lists of layers split at the first matching layer type
conv_layers,fc_layers = split_at(model, Convolution2D)

In [62]:
print ("model:", len(model.layers))
print ("conv_layers:", len(conv_layers))
print ("fc_layers:", len(fc_layers))

model: 38
conv_layers: 31
fc_layers: 7


In [64]:
model.output_shape
#(None, 1000)

(None, 1000)

In [65]:
model.summary()
#NB64*64 = 4096


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_4 (Lambda)                (None, 3, 224, 224)   0           lambda_input_4[0][0]             
____________________________________________________________________________________________________
zeropadding2d_40 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_4[0][0]                   
____________________________________________________________________________________________________
convolution2d_40 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_40[0][0]           
____________________________________________________________________________________________________
zeropadding2d_41 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_40[0][0]           
___________________________________________________________________________________________

In [83]:
conv_model = Sequential(conv_layers)

In [90]:
print (len(conv_model.layers))
print (conv_model.output_shape[1:])
#recall conv_model is model split at last Convolution2D layer
#can see shape of last Convolution2D layer in the model.summary

31
(512, 14, 14)


In [69]:
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)
trn_features = conv_model.predict_generator(batches, batches.nb_sample)

In [70]:
print ("val_features:", type(val_features), val_features.shape)
print ("trn_features:", type(trn_features), trn_features.shape)

val_features: <type 'numpy.ndarray'> (136, 512, 14, 14)
trn_features: <type 'numpy.ndarray'> (320, 512, 14, 14)


In [71]:
#NB: utils.get_data(path, target_size=(224,224))
#by default this uses array size 224 x 224. may want to look at other sizes for speed/accuracy optimisaiton
trn = get_data(WORKING_TRAIN)
val = get_data(WORKING_VALID)

Found 320 images belonging to 2 classes.
Found 136 images belonging to 2 classes.


In [72]:
#Finally, we can precompute the output of all but the last dropout and dense layers, for creating the first stage of the model:
model.pop()
model.pop()

In [73]:
print ("model:", len(model.layers))


model: 36


In [74]:
ll_val_feat = model.predict_generator(val_batches, val_batches.nb_sample)
ll_feat = model.predict_generator(batches, batches.nb_sample)
print ("ll_val_feat:", type(ll_val_feat), ll_val_feat.shape)
print ("ll_feat:", type(ll_feat), ll_feat.shape)

ll_val_feat: <type 'numpy.ndarray'> (136, 4096)
ll_feat: <type 'numpy.ndarray'> (320, 4096)


In [75]:
test = get_data(WORKING_TEST)
print ("test:", type(test), test.shape)

Found 306 images belonging to 1 classes.
test: <type 'numpy.ndarray'> (306, 3, 224, 224)


The functions automate creating a model that trains the last layer from scratch, and then adds those new layers on to the main model.

In [76]:
def get_ll_layers():
    #creates and returns 3 layers
    return [ 
        BatchNormalization(input_shape=(4096,)),
        Dropout(0.5),
        Dense(2, activation='softmax') 
        ]

In [77]:
def train_last_layer(i):
    ll_layers = get_ll_layers()
    ll_model = Sequential(ll_layers)
    ll_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    ll_model.optimizer.lr=1e-5
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), nb_epoch=12)
    ll_model.optimizer.lr=1e-7
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), nb_epoch=1)
    ll_model.save_weights(model_path+'ll_bn' + i + '.h5')

    vgg = Vgg16()
    model = vgg.model
    model.pop(); model.pop(); model.pop()
    for layer in model.layers: layer.trainable=False
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    ll_layers = get_ll_layers()
    for layer in ll_layers: model.add(layer)
    for l1,l2 in zip(ll_model.layers, model.layers[-3:]):
        l2.set_weights(l1.get_weights())
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.save_weights(model_path+'bn' + i + '.h5')
    return model


In [78]:
def get_conv_model(model):
    #
    layers = model.layers
    last_conv_idx = [index for index,layer in enumerate(layers) 
                         if type(layer) is Convolution2D][-1]
    #last_conv_idx = index position of last Convolution2D layer

    conv_layers = layers[:last_conv_idx+1]
    #all layers from start to last Convolution2D layer
    conv_model = Sequential(conv_layers)
    #conv_model = model composed of layers from start to last Convolution2D layer
    #conv_model = model less the last 7 layers. ie:  MaxPooling2D  and Flatten, Dense, Dropout, Dense, Dropout, Dense layers.
    fc_layers = layers[last_conv_idx+1:]
    #fc_layers = layers after last Convolution2D layer
    return conv_model, fc_layers, last_conv_idx

In [79]:
def get_fc_layers(p, in_shape):
    #creates new set of layers with dropout rate and shape specified
    return [
        MaxPooling2D(input_shape=in_shape),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(2, activation='softmax')
        ]

In [95]:

def train_dense_layers(i, model):
    conv_model, fc_layers, last_conv_idx = get_conv_model(model)
    conv_shape = conv_model.output_shape[1:]
    fc_model = Sequential(get_fc_layers(0.5, conv_shape))
    for l1,l2 in zip(fc_model.layers, fc_layers): 
        weights = l2.get_weights()
        l1.set_weights(weights)
    fc_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                     metrics=['accuracy'])
    fc_model.fit(trn_features, trn_labels, nb_epoch=2, 
         batch_size=batch_size, validation_data=(val_features, val_labels))

    gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.05, 
       width_zoom_range=0.05, zoom_range=0.05,
       channel_shift_range=10, height_shift_range=0.05, shear_range=0.05, horizontal_flip=True)
    batches = gen.flow(trn, trn_labels, batch_size=batch_size)
    val_batches = image.ImageDataGenerator().flow(val, val_labels, 
                      shuffle=False, batch_size=batch_size)

    for layer in conv_model.layers: layer.trainable = False
    for layer in get_fc_layers(0.5, conv_shape): conv_model.add(layer)
    for l1,l2 in zip(conv_model.layers[last_conv_idx+1:], fc_model.layers): 
        l1.set_weights(l2.get_weights())

    conv_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                       metrics=['accuracy'])
    conv_model.save_weights(model_path+'no_dropout_bn' + i + '.h5')
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=1, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)
    for layer in conv_model.layers[16:]: layer.trainable = True
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=8, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)

    conv_model.optimizer.lr = 1e-7
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=10, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)
    conv_model.save_weights(model_path + 'aug' + i + '.h5')

def train_dense_layers(i, model):
    #
    conv_model, fc_layers, last_conv_idx = get_conv_model(model)
    #
    
    conv_shape = conv_model.output_shape[1:]
    #conv_shape = shape of last Convolution2D layer
    fc_model = Sequential(get_fc_layers(0.5, conv_shape))
    #creates new set of layers with dropout rate and shape specified
    
    #fc_layers = list layers after last Convolution2D layer in model
    for l1,l2 in zip(fc_model.layers, fc_layers): 
        #copies weights from layers in original model to layers in new model
        weights = l2.get_weights()
        l1.set_weights(weights)
    
    fc_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    #compiling model required before can fit.
    
    fc_model.fit(trn_features, trn_labels, nb_epoch=2, batch_size=batch_size, validation_data=(val_features, val_labels))
    #only two epoch here. maybe increase in future and look @ history.

    #now do the augmentation
    gen = image.ImageDataGenerator(rotation_range=10, 
                                   width_shift_range=0.05, 
                                   width_zoom_range=0.05, 
                                   zoom_range=0.05, 
                                   channel_shift_range=10, 
                                   height_shift_range=0.05, 
                                   shear_range=0.05, 
                                   horizontal_flip=True)
    
    batches = gen.flow(trn, trn_labels, batch_size=batch_size)
    val_batches = image.ImageDataGenerator().flow(val, val_labels, shuffle=False, batch_size=batch_size)

    #set all layers in conv_model to non trainable
    for layer in conv_model.layers: 
        layer.trainable = False
    
    #set dropout rate and shape for new layers to be added to conv_model
    for layer in get_fc_layers(0.5, conv_shape): 
        conv_model.add(layer)
        
    #copy weights from layers in fc_model to conv_model.
    for l1,l2 in zip(conv_model.layers[last_conv_idx+1:], fc_model.layers): 
        l1.set_weights(l2.get_weights())

    #compile model required before it can be used
    conv_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                       metrics=['accuracy'])
    
    #
    conv_model.save_weights(model_path+'no_dropout_bn' + i + '.h5')
    
    #nb: at this stage, layers in conv_model are untrainable
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=1, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)
    
    #nb: previously made these layers untrainable, now make them trainable
    for layer in conv_model.layers[16:]: 
        layer.trainable = True
    
    #do some fitting, multiple epochs.
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=8, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)

    conv_model.optimizer.lr = 1e-7
    #set lower training lr value, do more epochs
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=10, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)
    
    #
    conv_model.save_weights(model_path + 'aug' + i + '.h5')

In [96]:
for i in range(5):
    i = str(i)
    model = train_last_layer(i)
    train_dense_layers(i, model)

Train on 320 samples, validate on 136 samples
Epoch 1/12
320/320 [==============================] - 0s - loss: 1.0892 - acc: 0.6031 - val_loss: 0.9481 - val_acc: 0.6029
Epoch 2/12
320/320 [==============================] - 0s - loss: 1.0221 - acc: 0.6031 - val_loss: 0.9318 - val_acc: 0.6029
Epoch 3/12
320/320 [==============================] - 0s - loss: 1.0349 - acc: 0.5750 - val_loss: 0.9172 - val_acc: 0.6029
Epoch 4/12
320/320 [==============================] - 0s - loss: 0.9606 - acc: 0.6219 - val_loss: 0.9051 - val_acc: 0.6103
Epoch 5/12
320/320 [==============================] - 0s - loss: 0.9864 - acc: 0.5656 - val_loss: 0.8959 - val_acc: 0.6103
Epoch 6/12
320/320 [==============================] - 0s - loss: 1.0501 - acc: 0.5406 - val_loss: 0.8898 - val_acc: 0.6250
Epoch 7/12
320/320 [==============================] - 0s - loss: 0.9569 - acc: 0.5906 - val_loss: 0.8834 - val_acc: 0.6324
Epoch 8/12
320/320 [==============================] - 0s - loss: 0.9545 - acc: 0.6188 - val_l

ValueError: You called `set_weights(weights)` on layer "batchnormalization_7" with a  weight list of length 0, but the layer was expecting 4 weights. Provided weights: []...